In [5]:
# Imports
from SDL import strategy,world
from SDL.utility import *

First, a "campaign" or list of campaigns will be created. A campaign consists of an instance of SDL.utility.CampaignInfo. It is a vessel for carrying all information necessary to perform a set of (V)SDL runs. It begines with building an environment. To do so, declare the relevant experiments instances from SDL.world. Check that module for the current options. Then, decide which environment to build. Current option is only VSDLEnvironment. Then, the ML agent will ne to be declared. Decide which to use from SDL.strategy, then feed it the necessary arguments (if deviating from defaults). Finally, you are ready to put it all together into a CampaignInfo instance. You can adjust some campaign arguments, like the number of runs to perform, the sampling procedure, the campaign name, etc.

In [9]:
# Create the experiments
experiments = {
    'BladeCoat': world.BladeCoat(action_space=['Stability'],inputs={'temperature':[0,400], 'time':[0,300]},cost=10.0),
    'Stability': world.Stability(cost=10.0),
}

# Create the environment
environment = world.VSDLEnvironment(experiments=experiments)

# Create the agent
agent = strategy.ArchitectureOne(epsilon=0.3)

# Combine into a campagin
campaign1 = CampaignInfo(
    name = 'Architecture Zero -- Environment 000',
    runs = 2,
    environment = environment,
    agent = agent,
    sampling_procedure = [
        ('number_of_initial_datapoints', 3),
        ('number_of_batches', 2),
        ('samples_per_batch', 2)
        ],
)

# Create another campaign
experiments = {
    'BladeCoat': world.BladeCoat(action_space=['Stability'],inputs={'temperature':[0,400], 'time':[0,300]},cost=10.0),
    'Stability': world.Stability(cost=10.0),
}
environment = world.VSDLEnvironment(experiments=experiments)
agent = strategy.ArchitectureOne(epsilon=0.7)
campaign2 = CampaignInfo(
    name = 'Architecture Zero -- Environment 001',
    runs = 2,
    environment = environment,
    agent = agent,
    sampling_procedure = [
        ('number_of_initial_datapoints', 2),
        ('number_of_batches', 3),
        ('samples_per_batch', 1)
        ],
)

# Compile into a campaign list
campaign_list = [campaign1,campaign2]

Next, the campaigns are run. If the number of campaigns and the number of samples to be performed are small, they can be run right in this notebook. If there are many samples, many runs, and/or many campaigns, consider running them using HPC. To do this, dump the campaign list into a binary file using pickle. Then, use the runSDL.py module in this repo to read and run the binary file.

In [10]:
# Run campaign in notebook
#for _ in campaign_list:
#    _.run()

# Dump to a binary input file, to be run on an HPC
dump_campaign_list(campaign_list, 'my_VSDL_inputs - {} - .pkl'.format(datetime.datetime.now()))

Next, read the output file (and optionally the MAE file) generated from a campaign. The data will be read in as an instance of SDL.utility.SDLOutputData.

In [11]:
# Read in output from a campaign
data = read_output('Architecture Zero -- Environment 000.out.txt')
for k,v in data.__dict__.items():
    print(k,v)

name Architecture Zero -- Environment 000
color #
calc_stability None
runs 2
sampling_procedure [('Initial Datapoints per Processing Variable', 3), ('Number of Batches', 2), ('Samples per Batch', 2)]
states {1: [[[<material.State object at 0x7fe918b63d00>, <material.State object at 0x7fe9786fd5e0>], [<material.State object at 0x7fe978310550>, <material.State object at 0x7fe978310fd0>], [<material.State object at 0x7fe978310040>, <material.State object at 0x7fe9783109d0>], [<material.State object at 0x7fe978310760>, <material.State object at 0x7fe918d15f40>], [<material.State object at 0x7fe988515880>, <material.State object at 0x7fe988515220>], [<material.State object at 0x7fe9885158e0>, <material.State object at 0x7fe988515460>], [<material.State object at 0x7fe9885155e0>, <material.State object at 0x7fe9885157f0>], [<material.State object at 0x7fe988515280>, <material.State object at 0x7fe988515cd0>], [<material.State object at 0x7fe988515970>, <material.State object at 0x7fe988515bb

In [19]:
blade_coat = world.BladeCoat(inputs={'temperature': [0, 400], 'time': [0, 300]})
reference_inputs = blade_coat.get_input_space()[1]
reference_targets = world.g1(reference_inputs[:,0]/400,p2=reference_inputs[:,1]/300)
cumulative_MAEs = np.array([
        get_cumulative_MAE(data,run,reference_inputs,reference_targets)
            for run in range(1,data.runs+1)])

In [22]:
cumulative_MAEs

array([[20.03307681, 73.09567002, 20.13515907, 73.08970196, 17.20177432,
        73.06187483, 16.38559755, 16.38559767, 73.04039472, 16.33327455,
        16.33327467]])